In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import seaborn as sns
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble        import RandomForestClassifier
from sklearn.linear_model        import LogisticRegression
from sklearn.svm        import SVC
from sklearn.preprocessing   import StandardScaler
from sklearn.model_selection import GroupKFold,ShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.inspection import permutation_importance

In [ ]:
def make_dataset(string):

    data = {}
    plant_parts =  np.sort(glob.glob(string))

    print("n. files: {:}".format(len(plant_parts)))

    for single_plant_parts in plant_parts:

        folder,data_type,body_part,stimulus,experiment = single_plant_parts.split('/')

        plant,stimulus,bodypart      = experiment[:-4].split('_')

        
        if plant not in data.keys():
            data[plant] = {'wrist':None,'tendril':[],'stimulus':stimulus,'plant':plant}

        print('\r\r\r',plant,stimulus,bodypart,end='')

        
        if 'wrist' in bodypart :

            df = pd.read_csv(single_plant_parts)
            data[plant]['wrist'] = df

        
        elif 'tendril' in bodypart :
            df = pd.read_csv(single_plant_parts)
            data[plant]['tendril'].append(df)

        else:
            print('bodypart error')

    # for each plant
    for plant in data.keys():
        
        
        tendril_x = pd.concat(data[plant]['tendril'],axis=1)['col_x']
        tendril_y = pd.concat(data[plant]['tendril'],axis=1)['col_y']
        tendril_z = pd.concat(data[plant]['tendril'],axis=1)['col_z']

        
        if type(data[plant]['wrist']) != type(None):

            # preprocesses the data
            data[plant]['all_bodyparts'] = {
                'time_idx':data[plant]['wrist'].index,
                'relative_time_idx':data[plant]['wrist'].index/data[plant]['wrist'].index.max(),
                'wrist_x':data[plant]['wrist']['col_x']-data[plant]['wrist']['col_x'][0],
                'wrist_y':data[plant]['wrist']['col_y']-data[plant]['wrist']['col_y'][0],
                'wrist_z':data[plant]['wrist']['col_z']-data[plant]['wrist']['col_z'][0],
                'tendril_mean_x' : tendril_x.mean(axis=1)-data[plant]['wrist']['col_x'],
                'tendril_mean_y' : tendril_y.mean(axis=1)-data[plant]['wrist']['col_y'],
                'tendril_mean_z' : tendril_z.mean(axis=1)-data[plant]['wrist']['col_z'],
                'tendril_std_x' : tendril_x.std(axis=1),
                'tendril_std_y' : tendril_y.std(axis=1),
                'tendril_std_z' : tendril_z.std(axis=1),
                'stimulus' : data[plant]['stimulus'],
                'plant_num' : data[plant]['plant']
            }

    
    df = pd.DataFrame([])
    for plant in data.keys():
        if type(data[plant]['wrist']) != type(None) :

            aux = pd.DataFrame(data[plant]['all_bodyparts'])

            aux['wrist'] = (aux[['wrist_x','wrist_y','wrist_z']]**2).sum(axis=1)**1/2

            aux['tendril'] = (aux[['tendril_mean_x','tendril_mean_y','tendril_mean_z']]**2).sum(axis=1)**1/2

            aux['d_wrist_x'] = aux['wrist_x'].diff()
            aux['d_wrist_y'] = aux['wrist_y'].diff()
            aux['d_wrist_z'] = aux['wrist_z'].diff()

            aux['d_tendril_mean_x'] = aux['tendril_mean_x'].diff()
            aux['d_tendril_mean_y'] = aux['tendril_mean_y'].diff()
            aux['d_tendril_mean_z'] = aux['tendril_mean_z'].diff()

            aux['d_tendril_std_x'] = aux['tendril_std_x'].diff()
            aux['d_tendril_std_y'] = aux['tendril_std_y'].diff()
            aux['d_tendril_std_z'] = aux['tendril_std_z'].diff()

            df = df.append(aux)

    return df

In [ ]:
string = 'Plant_dataset/coordinates/**/**/*.csv'
df     = make_dataset(string)
df['time_window'] = df['time_idx'].round(-2)
df['relative_time_window']  = df['relative_time_idx'].round(1)

In [ ]:
df[['time_idx','plant_num','stimulus']].groupby(['plant_num','stimulus']).max()

In [ ]:
time_cut    = 1505
time_cut_df = df[df['time_idx']<time_cut]

In [ ]:
extracted_features = pd.concat([time_cut_df.groupby(['plant_num','stimulus']).mean(),time_cut_df.groupby(['plant_num','stimulus']).std()],axis=1,keys=['mean','std']).reset_index(level=[0,1])
extracted_features.columns = ['_'.join(col) for col in extracted_features.columns]

In [ ]:
def analyse(extracted_features,features,model):#,get_feature_importance=False):

    X_not_norm = extracted_features[features]
    y          = extracted_features[('stimulus_')].values

    scaler          = StandardScaler().fit(X_not_norm)
    X               = scaler.transform(X_not_norm)

    rs = StratifiedShuffleSplit(n_splits=25, test_size=.25, random_state=0)

    acc                 = []
    feature_importances = []
    i = 0
    for train_index, test_index in rs.split(X,y):
        print('\r',i,' ',end='')
        i+=1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train,y_train)

        #if get_feature_importance :

        X_val, y_val = X_test,y_test
        r = permutation_importance(model, X_val, y_val,
                                    n_repeats=30,
                                    random_state=0)
        
        #feature_importances.append(model.feature_importances_)
        feature_importances.append(r['importances_mean'])

        y_test_pred = model.predict(X_test)

        acc.append(accuracy_score(y_test, y_test_pred))

    return acc,feature_importances

def plot_acc(accRF,accLR,accSVC):
    
    acc = [accRF,accLR,accSVC]
    fig,axs = plt.subplots(1,1,figsize=[5,5])
    axs.boxplot(acc)
    axs.set_xticks([1, 2, 3], ['RF', 'LR', 'SVC'])
    axs.grid(True)
    axs.set_ylabel('acc')
    axs.set_ylim([0,1.1])

    #fig,axs = plt.subplots(1,2,figsize=[10,5])
    #axs[1].bar(features,np.array(feature_importances).mean(axis=0))
    #_=plt.xticks(rotation=90)
    #axs[1].set_title(window)
    #plt.show()

def plot_fi(featRF,featLR,featSVC,features):

    plt.figure()

    dfRF  = pd.DataFrame({'features':np.repeat(features,25),'feature importance':np.array(featRF).flatten()})
    dfRF['model'] = 'RF'

    dfLR  = pd.DataFrame({'features':np.repeat(features,25),'feature importance':np.array(featLR).flatten()})
    dfLR['model'] = 'LR'

    dfSVC = pd.DataFrame({'features':np.repeat(features,25),'feature importance':np.array(featSVC).flatten()})
    dfSVC['model'] = 'SVC'

    df = pd.concat([dfRF,dfLR,dfSVC])

    sns.boxplot(data=df,y='feature importance',hue='features',x='model')

### All features

In [ ]:
features =[
 "mean_time_idx" ,
 "mean_relative_time_idx" ,
 "mean_wrist_x" ,
 "mean_wrist_y" ,
 "mean_wrist_z" ,
 "mean_tendril_mean_x" ,
 "mean_tendril_mean_y" ,
 "mean_tendril_mean_z" ,
 "mean_tendril_std_x" ,
 "mean_tendril_std_y" ,
 "mean_tendril_std_z" ,
 "mean_wrist" ,
 "mean_tendril" ,
 "mean_d_wrist_x" ,
 "mean_d_wrist_y" ,
 "mean_d_wrist_z" ,
 "mean_d_tendril_mean_x" ,
 "mean_d_tendril_mean_y" ,
 "mean_d_tendril_mean_z" ,
 "mean_d_tendril_std_x" ,
 "mean_d_tendril_std_y" ,
 "mean_d_tendril_std_z" ,
 "mean_relative_time_window",
 "std_time_idx" ,
 "std_relative_time_idx" ,
 "std_wrist_x" ,
 "std_wrist_y" ,
 "std_wrist_z" ,
 "std_tendril_mean_x" ,
 "std_tendril_mean_y" ,
 "std_tendril_mean_z" ,
 "std_tendril_std_x" ,
 "std_tendril_std_y" ,
 "std_tendril_std_z" ,
 "std_wrist" ,
 "std_tendril" ,
 "std_d_wrist_x" ,
 "std_d_wrist_y" ,
 "std_d_wrist_z" ,
 "std_d_tendril_mean_x" ,
 "std_d_tendril_mean_y" ,
 "std_d_tendril_mean_z" ,
 "std_d_tendril_std_x" ,
 "std_d_tendril_std_y" ,
 "std_d_tendril_std_z" ,
 "std_relative_time_window" ,
            ]

accRF,featRF    = analyse(extracted_features,features,RandomForestClassifier())
accLR,featLR    = analyse(extracted_features,features,LogisticRegression())
accSVC,featSVC  = analyse(extracted_features,features,SVC())

plot_acc(accRF,accLR,accSVC)

plot_fi(featRF,featLR,featSVC,features)